In [69]:
from pydub import AudioSegment
from pydub.utils import mediainfo
import numpy as np

In [104]:
SLICE_DURATION = 0.1
SAMPLING_RATE = 16000

In [105]:
def get_sliced_audio_samples(path, slice_duration):
    slice_duration_ms = slice_duration * 1000
    audio = AudioSegment.from_wav(path)
    audio = audio.set_frame_rate(SAMPLING_RATE).set_channels(1)
    slice_starts = np.arange(0, audio.duration_seconds - slice_duration, slice_duration, dtype=np.float32)
    slice_starts_ms = slice_starts * 1000
    slice_func = lambda start: audio[start:start + slice_duration_ms]
    audio_slices = np.vectorize(slice_func)(slice_starts_ms)
    decoder_func = lambda segment: segment.get_array_of_samples()
    return np.array([decoder_func(audio_slice) for audio_slice in audio_slices])

In [108]:
get_sliced_audio_samples("males/0.wav", SLICE_DURATION)

array([[  71,   63,   43, ..., -130, -176,  -89],
       [ 100,  436,  891, ..., -532, -611, -693],
       [-748, -806, -838, ...,  -47,  -34,   21],
       ...,
       [-322, -361, -181, ..., 3228, 2362, 1643],
       [ 934,  309,  143, ...,  489,  601,  473],
       [ 400,  360,  365, ...,    0,    0,    0]], dtype=int16)